## Forked from https://www.kaggle.com/code/abdurrafae/improved-code-interpretation

**Lewis:** the only changes in this notebook are those needed to run the original one with the new Kaggle evaluation API

In [1]:
import sys
print('python版本：',sys.version)

import pkg_resources

def get_package_version(package_name):
    try:
        version = pkg_resources.get_distribution(package_name).version
        return version
    except pkg_resources.DistributionNotFound:
        return "Package not found"
package_name = "torch"
version = get_package_version(package_name)
print(f"{package_name}版本：{version}")

import torch

cuda_version = torch.version.cuda
print("CUDA版本：", cuda_version)

python版本： 3.10.12 (main, Jul  5 2023, 18:54:27) [GCC 11.2.0]
torch版本：2.1.2


/tmp/ipykernel_8067/3712959345.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


CUDA版本： 12.1


In [2]:
## Forked From  https://kaggle.com/code/xiaoz259/pure-rng/notebook


# credits:
# https://www.kaggle.com/code/olyatsimboy/aimo-openmath-mistral-baseline
# https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama
# https://www.kaggle.com/code/thedrcat/aimo-mixtral-baseline

In [3]:
import time

NOTEBOOK_START_TIME = time.time()

In [4]:
import aimo
try:
    env = aimo.make_env()
except:
    pass
iter_test = env.iter_test()

TO-DO

Change temperature as the question goes longer
Change temperature based on question lenght

# Zero-shot MMOS-DeepSeekMath-7B with self-consistency and generated code reasoning evaluation

Self-consistency is a modification of the standard greedy decoding in reasoning pipelines via sampling several diverse answers followed by aggregation, e.g., most common answer ([SC-CoT paper](https://arxiv.org/pdf/2203.11171.pdf)).

In this kernel, we will consider MMOS-DeepSeekMath-7B RL-tuned backbone; in my experiments, this model produces more consistent code reasoning and the code block execution will allow us to decrease arithmetic hallucinations.

In [5]:
DEBUG = False

QUANT = False

if QUANT:
    from transformers import BitsAndBytesConfig
    quantization_config = BitsAndBytesConfig(
        load_in_4bit = True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

USE_PAST_KEY = True

In [6]:
%%time
if QUANT:
    !pip install -U ./input/accelerate-wheelwhl/accelerate-0.29.1-py3-none-any.whl -qq
    !pip install -U ./input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq


import torch
import gc
'''
accelerate execution a bit
'''
torch.backends.cuda.enable_mem_efficient_sdp(False)

from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    AutoConfig,
    StoppingCriteria,
    set_seed
)

import transformers
print(f"Transformers Version: {transformers.__version__}")
set_seed(42)

Transformers Version: 4.39.3
CPU times: user 223 ms, sys: 9.45 ms, total: 233 ms
Wall time: 233 ms


In [7]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True

# df = pd.read_csv('./input/ai-mathematical-olympiad-prize/test.csv')
# df.head()

In [8]:
# PRIVATE = True
df = pd.read_csv('./input/ai-mathematical-olympiad-prize/train.csv')
df.head()

,id,problem,answer
0,229ee8,"Let $k, l > 0$ be parameters. The parabola $y ...",52
1,246d26,Each of the three-digits numbers $111$ to $999...,250
2,2fc4ad,Let the `sparkle' operation on positive intege...,702
3,430b63,What is the minimum value of $5x^2+5y^2-8xy$ w...,800
4,5277ed,There exists a unique increasing geometric seq...,211


In [9]:
def naive_parse(answer):
    '''
    get the first consecutive numbers
    '''
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return ''.join(out)

In [10]:
import re
import sys
import subprocess

def return_last_print(output, n):
    lines = output.strip().split('\n')
    if lines:
        return lines[n]
    else:
        return ""

def process_code(code, return_shell_output=False):
    
    def repl(match):
        if "real" not in match.group():
            return "{}{}".format(match.group()[:-1], ', real=True)')
        else:
            return "{}{}".format(match.group()[:-1], ')')
    code = re.sub(r"symbols\([^)]+\)", repl, code)

    if return_shell_output:
        code = code.replace('\n', '\n    ')
            # Add a try...except block
        code = "\ntry:\n    from sympy import *\n{}\nexcept Exception as e:\n    print(e)\n    print('FAIL')\n".format(code)
    
    if not return_shell_output:
        print(code)
    with open('code.py', 'w') as fout:
        fout.write(code)
    
    batcmd = 'timeout 7 ' + sys.executable + ' code.py'
    try:
        shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
        return_value = return_last_print(shell_output, -1)
        print(shell_output)
        if return_shell_output:
            if return_value=='FAIL':
                CODE_STATUS = False
                return_value = return_last_print(shell_output, -2)
                if "not defined" in return_value:
                    return_value+='\nTry checking the formatting and imports'
            else:
                CODE_STATUS = True
            return return_value, CODE_STATUS  
        code_output = round(float(eval(return_value))) % 1000
    except Exception as e:
        print(e,'shell_output')
        code_output = -1
    
    if return_shell_output:
        if code_output==-1:
            CODE_STATUS = False
        else:
            CODE_STATUS = True
        return code_output, CODE_STATUS  
    
    
    return code_output


def process_text_output(output):
    result = output    
    try:
        result_output = re.findall(r'\\boxed\{(\d+)\}', result)

        print('BOXED', result_output)
        # if not len(result_output):
        #     result_output = naive_parse(result) # this is toxic
        # else:
        result_output = result_output[-1]

        print('BOXED FINAL', result_output)
        if not len(result_output):
            result_output = -1
        
        else:
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING TEXT')
        result_output = -1
    
    return result_output


In [11]:
torch.cuda.empty_cache()
gc.collect()

20

In [12]:
import re
import math
import random

from collections import defaultdict

# n_repetitions = 12 if PRIVATE else 4 # Original notebook had 22 but times out :(
n_repetitions = 8 if PRIVATE else 4 # Original notebook had 22 but times out :(

TOTAL_TOKENS = 2048 # if PRIVATE else 512
# TOTAL_TOKENS = 512 # if PRIVATE else 512


if PRIVATE:
    TIME_LIMIT = 31500
else:
    TIME_LIMIT = 1

In [13]:
#list number of cuda
n_gpus = torch.cuda.device_count()
device_i = 0
if n_gpus > 1:
    device_i = 1



if PRIVATE:

    MODEL_PATH = "./input/deepseek-math"#"/kaggle/input/gemma/transformers/7b-it/1"
    DEEP = True

    config = AutoConfig.from_pretrained(MODEL_PATH)
    config.gradient_checkpointing = True

    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

    device_map = [('model.embed_tokens', 0),
                 ('model.layers.0', 0),
                 ('model.layers.1', 0),
                 ('model.layers.2', 0),
                 ('model.layers.3', 0),
                 ('model.layers.4', 0),
                 ('model.layers.5', 0),
                 ('model.layers.6', 0),
                 ('model.layers.7', 0),
                 ('model.layers.8', 0),
                 ('model.layers.9', 0),
                 ('model.layers.10', 0),
                 ('model.layers.11', 0),
                 ('model.layers.12', 0),
                 ('model.layers.13', 0),
                 ('model.layers.14', 0),
                 ('model.layers.15', 0),
                 ('model.layers.16', 0),
                 ('model.layers.17', 0),
                 ('model.layers.18', 0),
                 ('model.layers.19', 0),
                 ('model.layers.20', 0),
                 ('model.layers.21', 0),
                 ('model.layers.22', device_i),
                 ('model.layers.23', device_i),
                 ('model.layers.24', device_i),
                 ('model.layers.25', device_i),
                 ('model.layers.26', device_i),
                 ('model.layers.27', device_i),
                 ('model.layers.28', device_i),
                 ('model.layers.29', device_i),
                 ('model.norm', device_i),
                 ('lm_head', device_i)]

    device_map = {ii:jj for (ii,jj) in device_map}

    if QUANT:
        from transformers import BitsAndBytesConfig
        quantization_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
        )
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map="sequential",
            torch_dtype="auto",
            trust_remote_code=True, 
            quantization_config=quantization_config,
            config=config
        )
    else:  
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_PATH,
            device_map=device_map,
            torch_dtype="auto",
            trust_remote_code=True,
            #quantization_config=quantization_config,
            config=config
        )
    
    pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map=device_map,
)
    from transformers import StoppingCriteriaList

    class StoppingCriteriaSub(StoppingCriteria):
        def __init__(self, stops = [], encounters=1):
            super().__init__()
            self.stops = [stop.to("cuda") for stop in stops]

        def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
            for stop in self.stops:
                last_token = input_ids[0][-len(stop):]
                if torch.all(torch.eq(stop,last_token)):
                    return True
            return False


    stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"] #,  
    stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])
    
    model.dtype, model.hf_device_map

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:

code = \
"""Below is a math problem you are to solve (positive numerical answer):
\"{}\"
To accomplish this, first determine a sympy-based approach for solving the problem by listing each step to take and what functions need to be called in each step. Be clear so even an idiot can follow your instructions, and remember, your final answer should be positive integer, not an algebraic expression!
Write the entire script covering all the steps (use comments and document it well) and print the result. After solving the problem, output the final numerical answer within \\boxed{}.

Approach:"""


cot = \
"""Below is a math problem you are to solve (positive numerical answer!):
\"{}\"
Analyze this problem and think step by step to come to a solution with programs. After solving the problem, output the final numerical answer within \\boxed{}.\n\n"""

# cot = \
# """Below is a math problem you are to solve (positive numerical answer!):
# \"{}\"
# Analyze this problem and think step by step to come to a solution. You may use python to assist with solving it. Output the final numerical answer within \\boxed{}.\n\n"""

# cot =\
# '''
# **Problem Statement**:
# \"{}\"

# **Request**:
# Imagine you are an experienced math tutor. Please provide a comprehensive solution to the IMO math problem stated above. Include detailed explanations of each step and employ necessary mathematical concepts and techniques. Highlight key insights or techniques that are crucial for understanding and solving this type of problem.
# '''


#     code = \
# """
# 假设你是IMO数学竞赛指导老师，以下是一个数学问题：
# \"{}\"
# \n
# 1. 请先把这个问题翻译成中文。
# 2. 接着，确定一个基于sympy，numpy，或者scipy的解题方法，列出每一步的操作。
# 3. 请编对应的python脚本，并打印结果。
# 4. 你的最终答案应该是正整数，而不是代数表达式！
# 5. 解决问题后，在\\boxed{}中输出最终的数值答案。

# 解：
# """
#     cot = \
# """
# 假设你是IMO数学竞赛指导老师，以下是一个数学问题：
# \"{}\"
# \n
# 1. 请先把这个问题翻译成中文。
# 2. 分析这个问题，明确问题的要求与条件，并确定解题思路。
# 3. 尝试一步步解决这个问题，在必要的地方列式计算。 
# 4. 必要时，可以通过python脚本来辅助计算。
# 5. 解决问题后，在\\boxed{}中输出最终的数值答案。

# 解：
# """


prompt_options = [code,cot]
# prompt_options = [code]


In [15]:
import re
from collections import defaultdict
from collections import Counter

from numpy.random import choice
import numpy as np
from IPython.utils import io
from tqdm.auto import tqdm

# tool_instruction = '\n\nPlease integrate natural language reasoning with programs to solve the above problem, and put your final numerical answer within \\boxed{}.\nNote that the intermediary calculations may be real numbers, but the final numercal answer would always be an integer.'


# temperature = 2.0
temperature = 1
# top_p = 1.0
top_p = 1.1


temperature_coding = 1 # code need to be strict, but still should have some creativity to get through difficult problems
top_p_coding = 1.1

   
total_results = {}
total_answers = {}
best_stats = {}
total_outputs = {}
question_type_counts = {}
starting_counts = (2,3)

all_captured = []

import datetime
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

all_answers = {}

for i, (test, sample_submission) in tqdm(enumerate(iter_test), total=10):
    # iterate through every problems in the environment
    # use a loop get the test row and the corresponding sample_submissions row

    with io.capture_output() as captured: # i capture the printouts

        print(f"Solving problem {i} ...")
        TIME_SPENT = time.time() - NOTEBOOK_START_TIME

        if TIME_SPENT>TIME_LIMIT: # submit the prediction if no time is left
            sample_submission['answer'] = 0
            env.predict(sample_submission)
            break
            
        for trial_j in tqdm(range(n_repetitions)):   

            problem = test['problem'].values[0]
            print(f"\n\n\nQUESTION {i} - {trial_j} - TIME_SPENT : {TIME_SPENT:.0f} secs")
            
            best, best_count = best_stats.get(i,(-1,-1))
            if best_count>np.sqrt(trial_j)+1:
                print("SKIPPING CAUSE ALREADY FOUND BEST")
                continue
                
            outputs = total_outputs.get(i,[])
            text_answers, code_answers = question_type_counts.get(i,starting_counts)
            results = total_results.get(i,[])
            answers = total_answers.get(i,[])
            
            for _ in range(5): # clean the cache
                torch.cuda.empty_cache()
                gc.collect()
                time.sleep(0.2)

            try:
                ALREADY_GEN = 0
                code_error = None
                code_error_count = 0
                code_output = -1

                counts = np.array([text_answers,code_answers])

                # draw = choice(prompt_options, 1,
                #             p=counts/counts.sum())
                draw = [prompt_options[trial_j%len(prompt_options)]] # alternating 

                initail_message = draw[0].format(problem,"{}")            
                prompt = f"User: {initail_message}"

                current_printed = len(prompt)
                print(f"{trial_j}_{prompt}\n")

                model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
                input_len = len(model_inputs['input_ids'][0])

                generation_output = model.generate(**model_inputs, 
                                                max_new_tokens=TOTAL_TOKENS-ALREADY_GEN,
                                                return_dict_in_generate=USE_PAST_KEY,
                                                do_sample = True,
                                                temperature = temperature,
                                                top_p = top_p,
                                                num_return_sequences=1,
                                                stopping_criteria = stopping_criteria)

                if USE_PAST_KEY:
                    output_ids = generation_output.sequences[0]
                else:
                    output_ids = generation_output[0]
                decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
                print(f"{decoded_output[current_printed:]}\n")
                current_printed += len(decoded_output[current_printed:])
                cummulative_code = ""
                
                
                stop_word_cond = False
                for stop_word in stop_words:
                    stop_word_cond = stop_word_cond or (decoded_output[-len(stop_word):]==stop_word)
                    
                
                while (stop_word_cond) and (ALREADY_GEN<(TOTAL_TOKENS)):

                    if (decoded_output[-len("```python"):]=="```python"):
                        temperature_inner=temperature_coding
                        top_p_inner = top_p_coding
                        prompt = decoded_output
                    else:
                        temperature_inner=temperature
                        top_p_inner = top_p
                        try:
                            if (decoded_output[-len("``````output"):]=="``````output"):
                                code_text = decoded_output.split('```python')[-1].split("``````")[0]
                            else:
                                code_text = decoded_output.split('```python')[-1].split("```")[0]
                            

                            cummulative_code+=code_text
                            code_output, CODE_STATUS = process_code(cummulative_code, return_shell_output=True)
                            print('CODE RESULTS', code_output)
                            
                            # TODO: try to add the following prompt:
                            # Senario: CODE RESULTS 888.888888888889
                            # The result is not a positive integer, so we must have made a mistake somewhere. Let's go back and check our work.


                            if code_error==code_output:
                                code_error_count+=1
                            else:
                                code_error=code_output
                                code_error_count = 0

                            if not CODE_STATUS:
                                cummulative_code = cummulative_code[:-len(code_text)]

                                if code_error_count>=1:
                                    print("REPEATED ERRORS")
                                    break

                        except Exception as e:
                            print(e)
                            print('ERROR PARSING CODE')
                            code_output = -1

                       

                        if code_output!=-1:
                            self_correcting_prompt = ''
                            # try:
                            #     if int(code_output) != code_output:
                            #         self_correcting_prompt="\nThe result is not a positive integer, so we must have made a mistake somewhere. Let's go back and check our work."
                            # except:
                            #     pass

                            if (decoded_output[-len(")\n```"):]==")\n```"):
                                prompt = decoded_output+'```output\n'+str(code_output)+'\n```\n'+self_correcting_prompt
                            else:
                                prompt = decoded_output+'\n'+str(code_output)+'\n```\n' +self_correcting_prompt
                        else:
                            break # dont fix it lol, it's useless
                            prompt = decoded_output + self_correcting_prompt
                            cummulative_code=""



                    model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
                    ALREADY_GEN =  len(model_inputs['input_ids'][0])-input_len

                    if USE_PAST_KEY:
                        old_values = generation_output.past_key_values
                    else:
                        old_values = None

                    generation_output = model.generate(**model_inputs, 
                                                    max_new_tokens=TOTAL_TOKENS-ALREADY_GEN, 
                                                    return_dict_in_generate=USE_PAST_KEY,
                                                    past_key_values=old_values,
                                                    do_sample = True,
                                                    temperature = temperature_inner,
                                                    top_p = top_p_inner,
                                                    num_return_sequences=1, 
                                                    stopping_criteria = stopping_criteria)

                    if USE_PAST_KEY:
                        output_ids = generation_output.sequences[0]
                    else:
                        output_ids = generation_output[0]
                    decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
                    print(f"\nINTERMEDIATE OUT :\n{decoded_output[current_printed:]}\n")
                    current_printed+=len(decoded_output[current_printed:])
                    
                    stop_word_cond = False
                    for stop_word in stop_words:
                        stop_word_cond = stop_word_cond or (decoded_output[-len(stop_word):]==stop_word)

                if USE_PAST_KEY:
                    output_ids = generation_output.sequences[0]
                else:
                    output_ids = generation_output[0]

                raw_output = tokenizer.decode(output_ids[input_len:], skip_special_tokens=True)
                #print(f"\n\nOutput :\n{raw_output}\n")                            
                result_output = process_text_output(raw_output)
                
                try:
                    code_output = round(float(eval(code_output))) % 1000
                except Exception as e:
                    print(e,'final_eval')
                    code_output = -1

            except Exception as e:
                print(e,"5")
                result_output, code_output = -1, -1

            if code_output!=-1:
                outputs.append(code_output)
                code_answers+=1

            if result_output!=-1:
                outputs.append(result_output)
                text_answers+=1

            if len(outputs) > 0:
                occurances = Counter(outputs).most_common()
                print(occurances)
                if occurances[0][1] > best_count:
                    print("GOOD ANSWER UPDATED!")
                    best = occurances[0][0]
                    best_count = occurances[0][1]
                if occurances[0][1] > 5:
                    print("ANSWER FOUND!")
                    break

            results.append(result_output)
            answers.append(code_output)
            
            best_stats[i] = (best, best_count) 
            question_type_counts[i] = (text_answers, code_answers)
            total_outputs[i] = outputs
            
            total_results[i] = results
            total_answers[i] = answers

            print("code_answers",code_answers-starting_counts[1],"text_answers",text_answers-starting_counts[0])
            if DEBUG:
                break
                
        print(f"Predicted best answer: {best_stats}")
        sample_submission['answer'] = best_stats[i][0]
        env.predict(sample_submission)

    cur_process = captured.stdout
    cur_process += '\n==sep==\n'
    with open(f'outputs_{i}.txt', 'w') as fh:
        fh.write(cur_process)
    all_captured.append(cur_process)


temperatures = '|'.join(map(str, [temperature, top_p, temperature_coding, top_p_coding]))
folder_name = f'all_captured_{timestamp}_T={int(TIME_SPENT)}_{n_repetitions}_{TOTAL_TOKENS}_{QUANT}_{temperatures}'

import os
os.makedirs(folder_name, exist_ok=True)
for i, captured in enumerate(all_captured):
    with open(f'{folder_name}/output_{i}.txt', 'w') as fh:
        fh.write(captured)
    # change to md
    os.rename(f'{folder_name}/output_{i}.txt', f'{folder_name}/output_{i}.md')

all_captured_txt = '\n'.join(all_captured)
with open(f'{folder_name}/all_outputs.txt', 'w') as fh:
    fh.write(all_captured_txt)


import shutil
import os
from IPython.display import display, Javascript

def save_and_copy_notebook(source_path, destination_path):
    # Save the notebook first
    display(Javascript('IPython.notebook.save_checkpoint();'))
    
    # Ensure the notebook is saved before copying
    import time
    time.sleep(3)  # Wait for a few seconds before copying

    # Copy the notebook
    shutil.copy2(source_path, destination_path)
    print(f"Notebook copied to {destination_path}")

# Usage example: specify the current notebook path and the target location
current_notebook_path = f'./updated-code-interpretation-n-repetitions-17.ipynb'
new_location_path = f'{folder_name}/notebook.ipynb'
save_and_copy_notebook(current_notebook_path, new_location_path)

  0%|          | 0/10 [00:00<?, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for

<IPython.core.display.Javascript object>

Notebook copied to all_captured_20240620-184323_T=735_8_2048_False_0.9|1.1|0.9|1.1/notebook.ipynb


In [16]:

# import os
# os.makedirs(folder_name, exist_ok=True)
# for i, captured in enumerate(all_captured):
#     with open(f'{folder_name}/output_{i}.txt', 'w') as fh:
#         fh.write(captured)
#     # change to md
#     os.rename(f'{folder_name}/output_{i}.txt', f'{folder_name}/output_{i}.md')

# all_captured = '\n'.join(all_captured)
# with open(f'{folder_name}/all_outputs.txt', 'w') as fh:
#     fh.write(all_captured)


# import shutil
# import os

# from IPython.display import display, Javascript

# def save_and_copy_notebook(source_path, destination_path):
#     # Save the notebook first
#     display(Javascript('IPython.notebook.save_checkpoint();'))
    
#     # Ensure the notebook is saved before copying
#     import time
#     time.sleep(3)  # Wait for a few seconds before copying

#     # Copy the notebook
#     shutil.copy2(source_path, destination_path)
#     print(f"Notebook copied to {destination_path}")

# # Usage example: specify the current notebook path and the target location
# current_notebook_path = f'./updated-code-interpretation-n-repetitions-17.ipynb'
# new_location_path = f'{folder_name}/notebook.ipynb'
# save_and_copy_notebook(current_notebook_path, new_location_path)

In [17]:
# if PRIVATE:
#     df['answer'] = [best_stats[ii][0] for ii in range(len(df))]
# else:
#     df['answer'] = 2

In [18]:
# df[['id','answer']].to_csv("foo.csv", header=True, index=False)

In [19]:
# if not PRIVATE:
#     df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
#     if PRIVATE:
#         df['model_answer'] = [best_stats[ii][0] for ii in range(len(df))]
#         df['match'] = df.answer == df.model_answer
#         print(f'{df.match.sum()} matches in {len(df)} examples')

In [20]:
with open('code.py', 'w') as fout:
    fout.write("print('done')")

batcmd = 'timeout 7 ' + sys.executable + ' code.py'
try:
    shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
    print(shell_output)
except:
    pass

done

